In [1]:
import numpy as np
import pandas as pd
import sys,os

def load_files(batch):
    images = []
    labels = []
    for i in batch:
        print('Loading File: ' + i)
        x = np.load(i,encoding = 'latin1',allow_pickle=True).item()
        keys = x.keys()
        for key in keys:
            images.append(x[key][0])
            labels.append(x[key][1])
    return np.array(images),np.array(labels)

def get_feature(labels,feature):
    feature_values = []
    for i in labels:
        feature_values.append(i[feature])
    feature_values = np.array(feature_values)
    return feature_values

def get_cos_values(zenith,azimuth):
    cos1 = []
    cos2 = []
    cos3 = []
    for i,j in zip(zenith,azimuth):
        cos1.append(np.sin(i) * np.cos(j))
        cos2.append(np.sin(i) * np.sin(j))
        cos3.append(np.cos(i))
    return np.array(cos1),np.array(cos2),np.array(cos3)

In [2]:
file_path = '/fs/scratch/PAS1495/amedina/'
y = os.listdir(file_path+'processed_3D')

file_names = []

for i in y:
    file_names.append(file_path+'processed_3D/'+i)

file_names_batched = list(np.array_split(file_names,60))

In [3]:
images1,labels = load_files(file_names_batched[0])

Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5958.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_207.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5774.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5256.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4768.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_2519.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4632.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5356.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4155.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5633.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4163.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_7039.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_6819.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_3831.npy
Loading

Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_7850.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5971.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_1227.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4001.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4573.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_2758.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_6983.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_3617.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_6790.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4963.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_705.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_7135.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_1638.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_1216.npy
Loading

In [4]:
images = images1[:,:,:,:,[0,7]]


zenith_values = get_feature(labels,1)
azimuth_values = get_feature(labels,2)

cos1,cos2,cos3 = get_cos_values(zenith_values,azimuth_values)

print(images.shape)

(5637, 60, 10, 19, 2)


In [5]:
import tensorflow as tf

import keras
from keras.models import Sequential,load_model, Model
from keras.layers import Dense, Dropout, Flatten, Input,InputLayer, Concatenate
from keras.layers import Conv3D, MaxPooling3D , GaussianNoise,BatchNormalization
from keras import backend as K
from sklearn.model_selection import train_test_split,KFold
from keras.layers import LeakyReLU,SpatialDropout3D,GaussianDropout,Lambda,UpSampling1D
from keras import regularizers

Using TensorFlow backend.


In [6]:
cos_values = np.array(list(zip(cos1,cos2,cos3)))
cos_values1 = np.zeros([len(cos_values),len(cos_values[0])])
for i in range(len(cos_values)):
    for j in range(len(cos_values[0])):
        cos_values1[i][j]=(cos_values[i][j] + 1.0)/2.0

In [7]:
def get_data(images,values):
    x_train, x_test , y_train , y_test = train_test_split(images,values,test_size = 0.1 , random_state=42)
    return x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test = get_data(images,cos_values1)

In [8]:
def loss_space_angle(y_true,y_pred):
    y_true1 = y_true*2.0-1.0
    y_pred1 = y_pred*2.0-1.0
    subtraction = tf.math.subtract(y_true1,y_pred1)
    y = tf.matrix_diag_part(K.dot(subtraction,K.transpose(subtraction)))
    loss = tf.math.reduce_mean(y)
    return loss


In [9]:
number_features = 2

img_heights,img_rows, img_cols = 60,10,19
input_shape = (img_rows, img_cols)

kernel = 3
kernel2 = 2

model1_input = Input(shape=(img_heights,img_rows,img_cols,number_features))

model1 = LeakyReLU(alpha = 0.01)(model1_input)
output1 = MaxPooling3D(kernel2,padding='same')(model1)
model1 = Conv3D(32,kernel,padding='same',kernel_regularizer=regularizers.l2(0.01))(output1)

model1 = LeakyReLU(alpha = 0.01)(model1)
output2 = MaxPooling3D(kernel2,padding='same')(model1)
model1 = Conv3D(32,kernel,padding='same',kernel_regularizer=regularizers.l2(0.01))(output2)

model1 = LeakyReLU(alpha = 0.01)(model1)
output3 = MaxPooling3D(kernel2,padding='same')(model1)
model1 = Conv3D(32,kernel,padding='same',kernel_regularizer=regularizers.l2(0.01))(output3)

cnn_model1 = Flatten()(model1_input)
cnn_model2 = Flatten()(model1)
cnn_model3 = Flatten()(output1)
cnn_model4 = Flatten()(output2)
cnn_model5 = Flatten()(output3)
cnn_model = Concatenate(axis=-1)([cnn_model1,
                                cnn_model2,
                                cnn_model3,
                                cnn_model4,
                                cnn_model5
                                ])

cnn_model = Model(inputs=model1_input,outputs=cnn_model)
opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-5, amsgrad=False)
cnn_model.compile(optimizer=opt , loss = loss_space_angle)

print(cnn_model.summary())

W0805 14:53:20.908092 47511200458048 deprecation_wrapper.py:119] From /users/PAS1495/amedina/.local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 14:53:20.921596 47511200458048 deprecation_wrapper.py:119] From /users/PAS1495/amedina/.local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 14:53:20.926548 47511200458048 deprecation_wrapper.py:119] From /users/PAS1495/amedina/.local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 14:53:20.995657 47511200458048 deprecation_wrapper.py:119] From /users/PAS1495/amedina/.local/lib/python2.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Opti

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 10, 19, 2 0                                            
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 60, 10, 19, 2 0           input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling3d_1 (MaxPooling3D)  (None, 30, 5, 10, 2) 0           leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 30, 5, 10, 32 1760        max_pooling3d_1[0][0]            
__________________________________________________________________________________________________
leaky_re_l

In [10]:
input_new = Input(shape=(img_heights,img_rows,img_cols,number_features))

output = Lambda(lambda x: cnn_model(x))(input_new)

model = Dropout(0.5)(output)
model = Dense(512)(model)
model = LeakyReLU(alpha = 0.01)(model)
model = Dropout(0.5)(model)
model = Dense(512)(model)
model = LeakyReLU(alpha = 0.01)(model)

input_new_prime = Flatten()(input_new)
model = Concatenate(axis=-1)([model, input_new_prime])

predictions = Dense(3)(model)

model = Model(inputs=input_new,outputs=predictions)
opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-5, amsgrad=False)
model.compile(optimizer=opt , loss = loss_space_angle)
    

W0805 14:53:21.170094 47511200458048 deprecation_wrapper.py:119] From /users/PAS1495/amedina/.local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0805 14:53:21.176040 47511200458048 deprecation.py:506] From /users/PAS1495/amedina/.local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
#keras.utils.plot_model(cnn_model, to_file='cnn_model.png', show_shapes=False, show_layer_names=True, rankdir='TB')
#keras.utils.plot_model(model, to_file='model.png', show_shapes=False, show_layer_names=True, rankdir='TB')

In [12]:
history1 = []
history2 = []
batch_numbers = [128]
epochs=40

Wsave = model.get_weights()

for batch_size in batch_numbers:
    with tf.Session(config=tf.ConfigProto(
                    device_count={'CPU': 1})) as sess:
        K.set_session(sess)
        sess.run(tf.global_variables_initializer())
        history=model.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(x_test, y_test))
    history_dict = history.history
    history1.append(history_dict['loss'])
    history2.append(history_dict['val_loss'])
    model.set_weights(Wsave)

W0805 14:53:21.264204 47511200458048 deprecation_wrapper.py:119] From /users/PAS1495/amedina/.local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



Train on 5073 samples, validate on 564 samples
Epoch 1/40
5073/5073 [==============================] - 6s 1ms/step - loss: 1.4932 - val_loss: 0.9512
Epoch 2/40
5073/5073 [==============================] - 5s 1ms/step - loss: 0.8125 - val_loss: 0.7390
Epoch 3/40
4352/5073 [========================>.....] - ETA: 0s - loss: 0.6622

KeyboardInterrupt: 

In [ ]:
#import matplotlib.pyplot as plt
#history_dict = history.history
#print(history_dict.keys())
#loss_values = history_dict['loss']
#val_loss_values = history_dict['val_loss']

#epochs = range(1, len(loss_values)+1)

#start = 2

#plt.plot(epochs[start:len(epochs)],loss_values[start:len(loss_values)],'bo',label='Training loss')
#plt.plot(epochs[start:len(epochs)],val_loss_values[start:len(val_loss_values)],'b',label='Validation loss')
#plt.title('Training and validation loss')
#plt.xlabel('Epochs')
#plt.ylabel('Loss')
#plt.legend()

#plt.show()

In [ ]:
import matplotlib.pyplot as plt
labels = ['16','32','64','128','256']
count = 0
count1 = 0
epochs = range(1, len(history1[0])+1)
plt.figure()
for i in history1:
    plt.plot(epochs,i,label = labels[count])
    count+=1
for j in history2:
    plt.plot(epochs,j,label=labels[count1])
    count1+=1
plt.legend()
plt.show()

In [ ]:
#plt.savefig('figure.png')